# Sparse Mean-Reverting Portfolios
## Implementation of d'Aspremont (2011) and Related Methods

This notebook implements advanced sparse decomposition algorithms for identifying small, mean-reverting portfolios in high-dimensional asset universes. The methods are based on:

**"Identifying Small Mean Reverting Portfolios"** - d'Aspremont (2011)

### Key Algorithms Implemented:
1. **Sparse PCA** - L1-regularized principal components
2. **Box & Tao Decomposition** - Robust PCA (Low-rank + Sparse + Noise)
3. **Hurst Exponent** - Mean-reversion validation via R/S analysis
4. **Sparse Cointegration** - Elastic Net for cointegrating portfolios

### Performance Features:
- **Rust-accelerated** implementations for computational efficiency
- Real-world market data application
- Integration with portfolio analytics framework
- Live trading signal generation

---

## 1. Mathematical Foundation

### 1.1 Sparse PCA

Traditional PCA finds components that maximize variance:
$$\max_{w} \quad w^T \Sigma w \quad \text{s.t.} \quad \|w\|_2 = 1$$

**Sparse PCA** adds an L1 penalty to induce sparsity:
$$\max_{w} \quad w^T \Sigma w - \lambda \|w\|_1 \quad \text{s.t.} \quad \|w\|_2 = 1$$

where:
- $\Sigma$ = covariance matrix of returns
- $w$ = portfolio weights
- $\lambda$ = sparsity parameter (larger → sparser)
- $\|w\|_1 = \sum_i |w_i|$ (L1 norm)

**Algorithm (Iterative Soft-Thresholding):**
1. Initialize $w$ = first eigenvector of $\Sigma$
2. Repeat until convergence:
   - $w_{new} = \Sigma w$
   - Apply soft thresholding: $w_i = \text{sign}(w_i) \cdot \max(|w_i| - \lambda, 0)$
   - Normalize: $w = w_{new} / \|w_{new}\|_2$

**Interpretation:** Sparse components identify a small subset of assets that capture most variance → easier to trade, lower transaction costs.

### 1.3 Hurst Exponent via R/S Analysis

The **Hurst exponent** $H$ characterizes long-term memory:
- $H = 0.5$: Random walk (Brownian motion)
- $H < 0.5$: **Mean-reverting** (anti-persistent) ← **Desired for trading!**
- $H > 0.5$: Trending (persistent)

**Rescaled Range (R/S) Statistic:**

For a time series $\{X_t\}$ and window size $n$:
1. Compute cumulative deviations: $Y_k = \sum_{i=1}^k (X_i - \bar{X})$
2. Range: $R(n) = \max_k Y_k - \min_k Y_k$
3. Standard deviation: $S(n) = \sqrt{\frac{1}{n}\sum_{i=1}^n (X_i - \bar{X})^2}$
4. Rescaled range: $\frac{R(n)}{S(n)}$

**Hurst Estimation:**
$$\mathbb{E}\left[\frac{R(n)}{S(n)}\right] \propto n^H$$

Taking logarithms:
$$\log(R/S) \approx H \cdot \log(n) + c$$

Estimate $H$ as the slope of $\log(R/S)$ vs $\log(n)$.

**Statistical Test:**
- $H_0$: $H = 0.5$ (random walk)
- $H_a$: $H < 0.5$ (mean-reverting)
- Reject $H_0$ if 95% CI upper bound $< 0.5$

In [ ]:
# Generate synthetic market data with mean-reverting structure
np.random.seed(42)

n_samples = 1000  # Time periods
n_assets = 20     # Assets

# Create factor structure: prices = common_factor + idiosyncratic + noise
# Common factor (market)
market_factor = np.cumsum(np.random.randn(n_samples) * 0.01)

# Idiosyncratic components (mean-reverting for some assets)
betas = np.random.uniform(0.5, 1.5, n_assets)
mean_revert_speed = np.random.uniform(0.05, 0.2, n_assets)
idiosyncratic = np.zeros((n_samples, n_assets))

for i in range(n_assets):
    # Ornstein-Uhlenbeck process for mean-reversion
    for t in range(1, n_samples):
        idiosyncratic[t, i] = (idiosyncratic[t-1, i] * (1 - mean_revert_speed[i]) + 
                               np.random.randn() * 0.005)

# Combine components
prices = 100 + market_factor[:, None] * betas + idiosyncratic * 10 + np.random.randn(n_samples, n_assets) * 0.5

# Create DataFrame
asset_names = [f'Asset{i:02d}' for i in range(n_assets)]
prices_df = pd.DataFrame(prices, columns=asset_names)
returns_df = prices_df.pct_change().dropna()

print(f"✓ Generated {n_samples} periods of {n_assets} assets")
print(f"  Price range: ${prices.min():.2f} - ${prices.max():.2f}")
print(f"  Returns: {returns_df.mean().mean()*252:.2%} annual return")
print(f"  Volatility: {returns_df.std().mean()*np.sqrt(252):.2%} annual vol")

# Display correlation matrix
fig = px.imshow(returns_df.corr(), 
                color_continuous_scale='RdBu_r',
                aspect='auto',
                title='Asset Return Correlations')
fig.show()

## 2. Visualizing Price Dynamics

Let's examine the generated price series and their statistical properties.

In [ ]:
# Plot sample price trajectories
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Sample Price Paths', 'Distribution of Returns', 
                    'Rolling Volatility (30-day)', 'Cumulative Returns'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# 1. Sample price paths
for i in range(5):
    fig.add_trace(
        go.Scatter(x=prices_df.index, y=prices_df.iloc[:, i], 
                   name=asset_names[i], mode='lines'),
        row=1, col=1
    )

# 2. Distribution of all returns
all_returns = returns_df.values.flatten()
fig.add_trace(
    go.Histogram(x=all_returns, nbinsx=50, name='Returns', showlegend=False),
    row=1, col=2
)

# 3. Rolling volatility
rolling_vol = returns_df.rolling(30).std() * np.sqrt(252)
for i in range(5):
    fig.add_trace(
        go.Scatter(x=rolling_vol.index, y=rolling_vol.iloc[:, i], 
                   name=asset_names[i], showlegend=False),
        row=2, col=1
    )

# 4. Cumulative returns
cum_returns = (1 + returns_df).cumprod() - 1
for i in range(5):
    fig.add_trace(
        go.Scatter(x=cum_returns.index, y=cum_returns.iloc[:, i] * 100, 
                   name=asset_names[i], showlegend=False),
        row=2, col=2
    )

fig.update_xaxes(title_text="Time Period", row=2, col=1)
fig.update_xaxes(title_text="Time Period", row=2, col=2)
fig.update_yaxes(title_text="Price ($)", row=1, col=1)
fig.update_yaxes(title_text="Frequency", row=1, col=2)
fig.update_yaxes(title_text="Ann. Vol (%)", row=2, col=1)
fig.update_yaxes(title_text="Cum. Return (%)", row=2, col=2)

fig.update_layout(height=800, showlegend=True, title_text="Market Data Overview")
fig.show()

print(f"\n📊 Market Statistics:")
print(f"  Mean daily return: {returns_df.mean().mean()*100:.4f}%")
print(f"  Median daily return: {returns_df.median().median()*100:.4f}%")
print(f"  Avg volatility: {returns_df.std().mean()*np.sqrt(252)*100:.2f}%")
print(f"  Skewness: {returns_df.apply(lambda x: x.skew()).mean():.3f}")
print(f"  Kurtosis: {returns_df.apply(lambda x: x.kurtosis()).mean():.3f}")

## 3. Sparse PCA Analysis

### Mathematical Deep Dive

The optimization problem for Sparse PCA can be reformulated as:

$$\min_{w} \quad -w^T \Sigma w + \lambda \|w\|_1 \quad \text{s.t.} \quad \|w\|_2^2 \leq 1$$

Using Lagrangian duality, this becomes:
$$\mathcal{L}(w, \mu) = -w^T \Sigma w + \lambda \|w\|_1 + \mu(\|w\|_2^2 - 1)$$

**Key Properties:**
1. **Sparsity-Variance Tradeoff**: As $\lambda \uparrow$, sparsity $\uparrow$ but explained variance $\downarrow$
2. **Convex Relaxation**: The problem is non-convex, but iterative soft-thresholding converges to a stationary point
3. **Cardinality**: Roughly $k \approx \frac{1}{\lambda}$ non-zero weights

**Statistical Interpretation:**
- **Before**: Traditional PCA weights are dense → hard to interpret, expensive to trade
- **After**: Sparse PCA → $k$ assets capture similar variance → practical portfolios

In [ ]:
# Apply Sparse PCA with multiple lambda values
lambdas = [0.01, 0.05, 0.1, 0.2, 0.5]
results_spca = {}

cov_matrix = returns_df.cov().values

for lam in lambdas:
    weights, explained_var, convergence = sparse_pca(
        cov_matrix, 
        lambda_param=lam, 
        max_iter=1000, 
        tol=1e-6
    )
    
    n_nonzero = np.sum(np.abs(weights) > 1e-6)
    results_spca[lam] = {
        'weights': weights,
        'explained_var': explained_var,
        'n_nonzero': n_nonzero,
        'convergence': convergence
    }
    
    print(f"λ = {lam:.2f}: {n_nonzero}/{n_assets} assets, "
          f"Explained Var = {explained_var:.4f}, "
          f"Converged in {convergence} iterations")

# Visualize sparsity pattern
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=[f'λ={lam:.2f} ({results_spca[lam]["n_nonzero"]} assets)' 
                    for lam in lambdas[:3]] + 
                   [f'λ={lam:.2f} ({results_spca[lam]["n_nonzero"]} assets)' 
                    for lam in lambdas[3:]] + ['Sparsity-Variance Tradeoff'],
    specs=[[{"type": "bar"}, {"type": "bar"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "bar"}, {"type": "scatter"}]]
)

# Plot weight distributions for each lambda
for idx, lam in enumerate(lambdas):
    row = 1 if idx < 3 else 2
    col = (idx % 3) + 1
    weights = results_spca[lam]['weights']
    
    fig.add_trace(
        go.Bar(x=asset_names, y=weights, name=f'λ={lam:.2f}', showlegend=False),
        row=row, col=col
    )

# Add tradeoff curve
sparsity = [results_spca[lam]['n_nonzero'] for lam in lambdas]
var_explained = [results_spca[lam]['explained_var'] for lam in lambdas]

fig.add_trace(
    go.Scatter(x=sparsity, y=var_explained, mode='lines+markers',
               marker=dict(size=10), name='Tradeoff', showlegend=False),
    row=2, col=3
)

# Add annotations for each point
for i, lam in enumerate(lambdas):
    fig.add_annotation(
        x=sparsity[i], y=var_explained[i],
        text=f'λ={lam:.2f}',
        showarrow=True, arrowhead=2,
        row=2, col=3
    )

fig.update_xaxes(title_text="# Non-zero Weights", row=2, col=3)
fig.update_yaxes(title_text="Explained Variance", row=2, col=3)
fig.update_layout(height=800, title_text="Sparse PCA: Sparsity Pattern Analysis")
fig.show()

## 4. Box & Tao Decomposition

### Robust PCA via Principal Component Pursuit

The nuclear norm minimization problem:
$$\min_{L,S} \quad \|L\|_* + \lambda \|S\|_1 \quad \text{s.t.} \quad \|X - L - S\|_F \leq \epsilon$$

Can be solved via ADMM with augmented Lagrangian:
$$\mathcal{L}_\rho(L, S, Y) = \|L\|_* + \lambda \|S\|_1 + \langle Y, X - L - S \rangle + \frac{\rho}{2}\|X - L - S\|_F^2$$

**ADMM Updates:**
1. **L-step** (SVD soft-thresholding):
   $$L^{k+1} = \arg\min_L \|L\|_* + \frac{\rho}{2}\|L - (X - S^k + Y^k/\rho)\|_F^2$$
   $$\Rightarrow L = U \cdot \mathcal{S}_{\frac{1}{\rho}}(\Sigma) \cdot V^T$$
   where $\mathcal{S}_\tau(\sigma) = \text{sign}(\sigma) \cdot \max(|\sigma| - \tau, 0)$

2. **S-step** (elementwise soft-thresholding):
   $$S^{k+1} = \mathcal{S}_{\frac{\lambda}{\rho}}(X - L^{k+1} + Y^k/\rho)$$

3. **Y-step** (dual update):
   $$Y^{k+1} = Y^k + \rho(X - L^{k+1} - S^{k+1})$$

**Convergence Condition:** $\|X - L^k - S^k\|_F / \|X\|_F < \epsilon$

In [ ]:
# Apply Box & Tao decomposition
price_matrix = prices_df.values
L, S, metrics = box_tao_decomposition(
    price_matrix, 
    lambda_param=0.1, 
    max_iter=500, 
    tol=1e-5
)

print(f"✓ Decomposition complete")
print(f"  Iterations: {metrics['iterations']}")
print(f"  Final error: {metrics['final_error']:.6f}")
print(f"  Rank of L: {np.linalg.matrix_rank(L, tol=1e-3)}")
print(f"  Sparsity of S: {np.sum(np.abs(S) > 1e-3)} / {S.size} ({100*np.sum(np.abs(S) > 1e-3)/S.size:.2f}%)")

# Visualize decomposition
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=('Original Matrix X', 'Low-Rank L (Common Factors)', 'Sparse S (Idiosyncratic)',
                    'X Sample Series', 'L Sample Series', 'S Sample Series'),
    specs=[[{"type": "heatmap"}, {"type": "heatmap"}, {"type": "heatmap"}],
           [{"type": "scatter"}, {"type": "scatter"}, {"type": "scatter"}]]
)

# Heatmaps
fig.add_trace(go.Heatmap(z=price_matrix[:100, :10].T, colorscale='RdBu', showscale=False), row=1, col=1)
fig.add_trace(go.Heatmap(z=L[:100, :10].T, colorscale='RdBu', showscale=False), row=1, col=2)
fig.add_trace(go.Heatmap(z=S[:100, :10].T, colorscale='RdBu', showscale=True), row=1, col=3)

# Time series for first asset
fig.add_trace(go.Scatter(y=price_matrix[:, 0], mode='lines', name='X', showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(y=L[:, 0], mode='lines', name='L', showlegend=False), row=2, col=2)
fig.add_trace(go.Scatter(y=S[:, 0], mode='lines', name='S', showlegend=False), row=2, col=3)

fig.update_layout(height=800, title_text="Box & Tao Decomposition: X = L + S")
fig.show()

# Analyze sparse component
print(f"\n📊 Sparse Component Analysis:")
print(f"  Mean absolute value: {np.mean(np.abs(S)):.4f}")
print(f"  Std absolute value: {np.std(np.abs(S)):.4f}")
print(f"  Max absolute value: {np.max(np.abs(S)):.4f}")

# Find assets with strongest idiosyncratic behavior
asset_sparsity = np.sum(np.abs(S) > np.percentile(np.abs(S), 90), axis=0)
top_assets_idx = np.argsort(asset_sparsity)[-5:]
print(f"\n  Top 5 assets with idiosyncratic behavior:")
for idx in top_assets_idx[::-1]:
    print(f"    {asset_names[idx]}: {asset_sparsity[idx]} significant deviations")

## 5. Hurst Exponent Analysis

### Advanced R/S Methodology

For a robust estimate, we compute the Hurst exponent using multiple lag windows:
$$\text{Lags: } \{8, 16, 32, 64, 128, 256\}$$

For each lag $n$:
1. **Segment the data** into non-overlapping windows of size $n$
2. **For each window:**
   $$\bar{X}_n = \frac{1}{n}\sum_{i=1}^n X_i$$
   $$Y_k = \sum_{i=1}^k (X_i - \bar{X}_n), \quad k = 1, \ldots, n$$
   $$R_n = \max_k Y_k - \min_k Y_k$$
   $$S_n = \sqrt{\frac{1}{n}\sum_{i=1}^n (X_i - \bar{X}_n)^2}$$
3. **Average** $(R/S)_n$ across all windows
4. **Regression:** $\log(R/S)_n = H \cdot \log(n) + c$

**95% Confidence Interval:**
$$H \pm 1.96 \cdot \text{SE}(H)$$

where $\text{SE}(H)$ is estimated from the regression standard error.

**Decision Rule:**
- If $\text{CI}_{\text{upper}} < 0.5$: **Mean-reverting** ✓
- If $\text{CI}_{\text{lower}} > 0.5$: **Trending**
- Otherwise: **Inconclusive**

In [ ]:
# Compute Hurst exponent for all assets
lags = [8, 16, 32, 64, 128, 256]
hurst_results = []

for i, asset in enumerate(asset_names):
    prices_series = prices_df[asset].values
    H, ci_lower, ci_upper, rs_values = hurst_exponent(prices_series, lags=lags)
    
    is_meanrev = ci_upper < 0.5
    classification = 'Mean-Reverting' if is_meanrev else ('Trending' if ci_lower > 0.5 else 'Uncertain')
    
    hurst_results.append({
        'Asset': asset,
        'H': H,
        'CI_Lower': ci_lower,
        'CI_Upper': ci_upper,
        'Classification': classification
    })

hurst_df = pd.DataFrame(hurst_results).sort_values('H')

# Display results
print("📈 Hurst Exponent Analysis Results:")
print("=" * 70)
print(hurst_df.to_string(index=False))
print("\n" + "=" * 70)

mean_rev_count = np.sum(hurst_df['CI_Upper'] < 0.5)
trending_count = np.sum(hurst_df['CI_Lower'] > 0.5)
print(f"\n  Mean-Reverting: {mean_rev_count}/{n_assets} ({100*mean_rev_count/n_assets:.1f}%)")
print(f"  Trending: {trending_count}/{n_assets} ({100*trending_count/n_assets:.1f}%)")
print(f"  Uncertain: {n_assets - mean_rev_count - trending_count}/{n_assets}")

# Visualize Hurst exponents
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Hurst Exponents with 95% CI', 'Distribution of Hurst Exponents',
                    'Sample R/S Analysis (Asset00)', 'Mean-Reversion Score'),
    specs=[[{"type": "scatter"}, {"type": "histogram"}],
           [{"type": "scatter"}, {"type": "bar"}]]
)

# 1. Hurst with confidence intervals
colors = ['green' if ci_u < 0.5 else ('red' if ci_l > 0.5 else 'orange') 
          for ci_l, ci_u in zip(hurst_df['CI_Lower'], hurst_df['CI_Upper'])]

fig.add_trace(
    go.Scatter(x=hurst_df['Asset'], y=hurst_df['H'], 
               mode='markers', marker=dict(size=10, color=colors),
               name='Hurst', showlegend=False),
    row=1, col=1
)

# Error bars for CI
for idx, row in hurst_df.iterrows():
    fig.add_trace(
        go.Scatter(x=[row['Asset'], row['Asset']], 
                   y=[row['CI_Lower'], row['CI_Upper']],
                   mode='lines', line=dict(color=colors[idx], width=2),
                   showlegend=False),
        row=1, col=1
    )

# Add H=0.5 reference line
fig.add_hline(y=0.5, line_dash="dash", line_color="black", row=1, col=1)

# 2. Histogram
fig.add_trace(
    go.Histogram(x=hurst_df['H'], nbinsx=20, name='Distribution', showlegend=False),
    row=1, col=2
)
fig.add_vline(x=0.5, line_dash="dash", line_color="red", row=1, col=2)

# 3. R/S plot for first asset (demonstration)
prices_sample = prices_df.iloc[:, 0].values
_, _, _, rs_vals = hurst_exponent(prices_sample, lags=lags)
log_lags = np.log(lags)
log_rs = np.log(rs_vals)

# Regression line
slope, intercept = np.polyfit(log_lags, log_rs, 1)
fitted_line = slope * log_lags + intercept

fig.add_trace(
    go.Scatter(x=log_lags, y=log_rs, mode='markers', 
               name='R/S', marker=dict(size=10), showlegend=False),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=log_lags, y=fitted_line, mode='lines',
               name=f'H={slope:.3f}', showlegend=True),
    row=2, col=1
)

# 4. Mean-reversion score (0.5 - H)
meanrev_score = 0.5 - hurst_df['H']
fig.add_trace(
    go.Bar(x=hurst_df['Asset'], y=meanrev_score,
           marker=dict(color=meanrev_score, colorscale='RdYlGn'),
           showlegend=False),
    row=2, col=2
)

fig.update_xaxes(tickangle=45, row=1, col=1)
fig.update_xaxes(tickangle=45, row=2, col=2)
fig.update_xaxes(title_text="log(lag)", row=2, col=1)
fig.update_yaxes(title_text="Hurst Exponent", row=1, col=1)
fig.update_yaxes(title_text="log(R/S)", row=2, col=1)
fig.update_yaxes(title_text="MR Score", row=2, col=2)

fig.update_layout(height=900, title_text="Hurst Exponent Analysis")
fig.show()

## 6. Sparse Cointegration via Elastic Net

### Cointegration Theory

Two price series $P_1(t)$ and $P_2(t)$ are **cointegrated** if:
$$P_1(t) - \beta P_2(t) = \epsilon(t)$$

where $\epsilon(t)$ is **stationary** (mean-reverting).

**Generalization to N assets:**
$$\sum_{i=1}^N w_i P_i(t) = \epsilon(t) \quad \text{(stationary)}$$

### Elastic Net Formulation

We seek sparse weights $w$ such that the portfolio is stationary:
$$\min_{w} \quad \sum_{t=1}^T \left(\sum_{i=1}^N w_i P_i(t) - \bar{P}_w\right)^2 + \lambda_1 \|w\|_1 + \lambda_2 \|w\|_2^2$$

where:
- **L1 penalty** $\lambda_1 \|w\|_1$: Induces sparsity
- **L2 penalty** $\lambda_2 \|w\|_2^2$: Encourages smoothness and handles multicollinearity

**Why Elastic Net?**
1. Pure LASSO ($\lambda_2 = 0$): Can select only $\min(n, T)$ variables
2. Pure Ridge ($\lambda_1 = 0$): Dense solution
3. **Elastic Net**: Best of both worlds

### Augmented Dickey-Fuller Test

To verify stationarity of portfolio $\sum w_i P_i(t)$:
$$\Delta y_t = \alpha + \beta t + \gamma y_{t-1} + \sum_{i=1}^p \delta_i \Delta y_{t-i} + \epsilon_t$$

**Null hypothesis:** $H_0: \gamma = 0$ (unit root, non-stationary)
**Alternative:** $H_a: \gamma < 0$ (stationary)

If $p\text{-value} < 0.05$: Reject $H_0$ → Portfolio is mean-reverting ✓

In [ ]:
# Apply sparse cointegration
weights, adf_stat, adf_pval, half_life = sparse_cointegration(
    prices_df.values,
    l1_ratio=0.7,  # Elastic Net mixing (0.7 = more L1)
    alpha=0.1,     # Overall regularization strength
    max_assets=10  # Maximum portfolio size
)

print(f"✓ Sparse Cointegration Portfolio")
print(f"  Portfolio size: {np.sum(np.abs(weights) > 1e-6)} assets")
print(f"  ADF Statistic: {adf_stat:.4f}")
print(f"  ADF p-value: {adf_pval:.6f}")
print(f"  Half-life: {half_life:.2f} periods")

is_stationary = adf_pval < 0.05
print(f"  Stationarity: {'✓ PASS' if is_stationary else '✗ FAIL'} (α=0.05)")

# Display portfolio weights
weights_df = pd.DataFrame({
    'Asset': asset_names,
    'Weight': weights
}).sort_values('Weight', key=abs, ascending=False)

print("\n📊 Portfolio Composition:")
print(weights_df[weights_df['Weight'].abs() > 1e-6].to_string(index=False))

# Construct portfolio value
portfolio_value = prices_df.values @ weights
portfolio_returns = np.diff(portfolio_value) / portfolio_value[:-1]

# Visualize portfolio behavior
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Portfolio Weights', 'Portfolio Price Path',
                    'Mean Reversion (Z-score)', 'Return Distribution'),
    specs=[[{"type": "bar"}, {"type": "scatter"}],
           [{"type": "scatter"}, {"type": "histogram"}]]
)

# 1. Weights
active_weights = weights_df[weights_df['Weight'].abs() > 1e-6]
fig.add_trace(
    go.Bar(x=active_weights['Asset'], y=active_weights['Weight'],
           marker=dict(color=['green' if w > 0 else 'red' for w in active_weights['Weight']]),
           showlegend=False),
    row=1, col=1
)

# 2. Portfolio path
fig.add_trace(
    go.Scatter(y=portfolio_value, mode='lines', name='Portfolio', showlegend=False),
    row=1, col=2
)
# Add mean line
mean_val = np.mean(portfolio_value)
fig.add_hline(y=mean_val, line_dash="dash", line_color="red", row=1, col=2)

# 3. Z-score (standardized deviations from mean)
z_score = (portfolio_value - mean_val) / np.std(portfolio_value)
fig.add_trace(
    go.Scatter(y=z_score, mode='lines', name='Z-score', 
               line=dict(color='blue'), showlegend=False),
    row=2, col=1
)
# Add ±2 std bands
fig.add_hline(y=2, line_dash="dash", line_color="red", row=2, col=1)
fig.add_hline(y=-2, line_dash="dash", line_color="red", row=2, col=1)
fig.add_hline(y=0, line_dash="solid", line_color="black", row=2, col=1)

# 4. Return distribution
fig.add_trace(
    go.Histogram(x=portfolio_returns, nbinsx=50, showlegend=False),
    row=2, col=2
)

fig.update_xaxes(tickangle=45, row=1, col=1)
fig.update_yaxes(title_text="Weight", row=1, col=1)
fig.update_yaxes(title_text="Price", row=1, col=2)
fig.update_yaxes(title_text="Std Deviations", row=2, col=1)
fig.update_yaxes(title_text="Frequency", row=2, col=2)

fig.update_layout(height=800, title_text="Sparse Cointegration Portfolio Analysis")
fig.show()

# Trading signal statistics
crossing_count = np.sum(np.abs(np.diff(np.sign(z_score))) > 0)
print(f"\n📈 Trading Signal Analysis:")
print(f"  Mean crossings: {crossing_count} times")
print(f"  Avg time between crossings: {len(z_score)/crossing_count:.1f} periods")
print(f"  Time above +2σ: {100*np.sum(z_score > 2)/len(z_score):.2f}%")
print(f"  Time below -2σ: {100*np.sum(z_score < -2)/len(z_score):.2f}%")

## 7. Real-World Portfolio Selection & Risk Management

### Multi-Criteria Portfolio Ranking

We combine multiple signals to rank candidate portfolios:

1. **Mean-Reversion Strength** (from Hurst exponent):
   $$\text{MR\_Score} = \max(0.5 - H, 0)$$

2. **Cointegration Quality** (from ADF test):
   $$\text{Coint\_Score} = -\log_{10}(p\text{-value})$$

3. **Sparsity** (transaction cost proxy):
   $$\text{Sparse\_Score} = 1 - \frac{\text{# non-zero weights}}{N}$$

4. **Half-Life** (speed of mean-reversion):
   $$\text{Speed\_Score} = \frac{1}{1 + \text{half-life}/10}$$

5. **Sharpe Ratio** (risk-adjusted returns):
   $$\text{Sharpe} = \frac{\mathbb{E}[r]}{\sigma(r)} \cdot \sqrt{252}$$

**Composite Score:**
$$\text{Total\_Score} = w_1 \cdot \text{MR} + w_2 \cdot \text{Coint} + w_3 \cdot \text{Sparse} + w_4 \cdot \text{Speed} + w_5 \cdot \text{Sharpe}$$

Default weights: $w = [0.25, 0.25, 0.15, 0.15, 0.20]$

### Risk Metrics

1. **Maximum Drawdown:**
   $$\text{MDD} = \max_{t} \left(\frac{\max_{s \leq t} V(s) - V(t)}{\max_{s \leq t} V(s)}\right)$$

2. **Value at Risk (VaR)** at 95% confidence:
   $$\text{VaR}_{0.95} = -\text{Quantile}_{0.05}(\text{returns})$$

3. **Expected Shortfall (CVaR):**
   $$\text{CVaR}_{0.95} = \mathbb{E}[\text{return} \mid \text{return} < -\text{VaR}_{0.95}]$$

4. **Calmar Ratio:**
   $$\text{Calmar} = \frac{\text{Annual Return}}{\text{MDD}}$$

In [ ]:
# Generate multiple portfolio candidates using different methods
portfolio_candidates = []

# 1. Sparse PCA portfolios (different lambdas)
for lam in [0.05, 0.1, 0.2]:
    w, _, _ = sparse_pca(cov_matrix, lambda_param=lam, max_iter=1000, tol=1e-6)
    portfolio_candidates.append({
        'method': 'SparsePCA',
        'params': f'λ={lam}',
        'weights': w
    })

# 2. Cointegration portfolios (different regularizations)
for l1_ratio in [0.5, 0.7, 0.9]:
    w, adf_s, adf_p, hl = sparse_cointegration(
        prices_df.values, l1_ratio=l1_ratio, alpha=0.1, max_assets=10
    )
    portfolio_candidates.append({
        'method': 'Cointegration',
        'params': f'L1={l1_ratio:.1f}',
        'weights': w,
        'adf_stat': adf_s,
        'adf_pval': adf_p,
        'half_life': hl
    })

# 3. Box & Tao sparse components (select assets with highest sparse component variance)
asset_sparse_var = np.var(S, axis=0)
top_k = 8
top_assets = np.argsort(asset_sparse_var)[-top_k:]
w_box_tao = np.zeros(n_assets)
w_box_tao[top_assets] = 1.0 / top_k  # Equal weight
portfolio_candidates.append({
    'method': 'BoxTao',
    'params': f'top-{top_k}',
    'weights': w_box_tao
})

# 4. Hurst-based portfolios (select most mean-reverting assets)
meanrev_assets = hurst_df.nsmallest(8, 'H')['Asset'].values
w_hurst = np.zeros(n_assets)
for asset in meanrev_assets:
    idx = asset_names.index(asset)
    w_hurst[idx] = 1.0 / len(meanrev_assets)
portfolio_candidates.append({
    'method': 'Hurst',
    'params': 'top-8',
    'weights': w_hurst
})

print(f"✓ Generated {len(portfolio_candidates)} portfolio candidates")

In [ ]:
# Evaluate each portfolio candidate
def calculate_risk_metrics(returns):
    """Calculate comprehensive risk metrics"""
    # Basic stats
    mean_return = np.mean(returns) * 252  # Annualized
    volatility = np.std(returns) * np.sqrt(252)
    sharpe = mean_return / volatility if volatility > 0 else 0
    
    # Drawdown
    cum_returns = np.cumprod(1 + returns)
    running_max = np.maximum.accumulate(cum_returns)
    drawdown = (cum_returns - running_max) / running_max
    max_drawdown = np.min(drawdown)
    
    # Calmar ratio
    calmar = mean_return / abs(max_drawdown) if max_drawdown != 0 else 0
    
    # VaR and CVaR
    var_95 = -np.percentile(returns, 5)
    cvar_95 = -np.mean(returns[returns < -var_95]) if np.any(returns < -var_95) else 0
    
    return {
        'annual_return': mean_return,
        'volatility': volatility,
        'sharpe': sharpe,
        'max_drawdown': max_drawdown,
        'calmar': calmar,
        'var_95': var_95,
        'cvar_95': cvar_95
    }

portfolio_results = []

for idx, pf in enumerate(portfolio_candidates):
    w = pf['weights']
    
    # Portfolio value and returns
    pf_value = prices_df.values @ w
    pf_returns = np.diff(pf_value) / pf_value[:-1]
    
    # Risk metrics
    metrics = calculate_risk_metrics(pf_returns)
    
    # Hurst exponent of portfolio
    H, ci_l, ci_u, _ = hurst_exponent(pf_value, lags=[8, 16, 32, 64, 128])
    mr_score = max(0.5 - H, 0)
    
    # ADF test (cointegration quality)
    from statsmodels.tsa.stattools import adfuller
    try:
        adf_result = adfuller(pf_value, maxlag=10)
        adf_pval = adf_result[1]
        coint_score = -np.log10(max(adf_pval, 1e-10))
    except:
        adf_pval = 1.0
        coint_score = 0
    
    # Half-life estimation
    pf_mean = np.mean(pf_value)
    deviations = pf_value - pf_mean
    lagged_dev = deviations[:-1]
    current_dev = deviations[1:]
    
    if len(lagged_dev) > 0 and np.std(lagged_dev) > 1e-6:
        theta = np.polyfit(lagged_dev, current_dev, 1)[0]
        half_life = -np.log(2) / np.log(abs(theta)) if 0 < abs(theta) < 1 else 999
    else:
        half_life = 999
    
    speed_score = 1 / (1 + half_life / 10)
    
    # Sparsity
    n_nonzero = np.sum(np.abs(w) > 1e-6)
    sparse_score = 1 - n_nonzero / n_assets
    
    # Composite score (weighted sum of normalized metrics)
    composite_score = (
        0.25 * mr_score / 0.5 +  # Normalize to [0, 1]
        0.25 * min(coint_score / 5, 1) +  # Cap at 5
        0.15 * sparse_score +
        0.15 * speed_score +
        0.20 * min(max(metrics['sharpe'], 0) / 2, 1)  # Cap Sharpe at 2
    )
    
    portfolio_results.append({
        'Portfolio': f"{pf['method']}_{pf['params']}",
        'Method': pf['method'],
        'N_Assets': n_nonzero,
        'Hurst': H,
        'MR_Score': mr_score,
        'ADF_pval': adf_pval,
        'Coint_Score': coint_score,
        'Half_Life': half_life,
        'Speed_Score': speed_score,
        'Sparse_Score': sparse_score,
        'Annual_Return': metrics['annual_return'],
        'Volatility': metrics['volatility'],
        'Sharpe': metrics['sharpe'],
        'Max_DD': metrics['max_drawdown'],
        'Calmar': metrics['calmar'],
        'VaR_95': metrics['var_95'],
        'CVaR_95': metrics['cvar_95'],
        'Composite_Score': composite_score
    })

results_df = pd.DataFrame(portfolio_results).sort_values('Composite_Score', ascending=False)

print("\n" + "="*100)
print("🏆 PORTFOLIO RANKING (by Composite Score)")
print("="*100)
print(results_df[['Portfolio', 'N_Assets', 'Hurst', 'Sharpe', 'Max_DD', 'Composite_Score']].to_string(index=False))
print("="*100)

In [ ]:
# Visualize portfolio comparison
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=('Composite Score Ranking', 'Risk-Return Profile',
                    'Mean-Reversion vs Sharpe', 'Drawdown Comparison',
                    'Portfolio Sparsity', 'Half-Life Distribution'),
    specs=[[{"type": "bar"}, {"type": "scatter"}],
           [{"type": "scatter"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "histogram"}]]
)

# 1. Composite scores
fig.add_trace(
    go.Bar(x=results_df['Portfolio'], y=results_df['Composite_Score'],
           marker=dict(color=results_df['Composite_Score'], colorscale='Viridis'),
           showlegend=False),
    row=1, col=1
)

# 2. Risk-Return scatter
colors_method = {'SparsePCA': 'red', 'Cointegration': 'blue', 'BoxTao': 'green', 'Hurst': 'orange'}
for method in results_df['Method'].unique():
    subset = results_df[results_df['Method'] == method]
    fig.add_trace(
        go.Scatter(x=subset['Volatility']*100, y=subset['Annual_Return']*100,
                   mode='markers', marker=dict(size=12),
                   name=method, text=subset['Portfolio']),
        row=1, col=2
    )

# 3. Mean-reversion vs Sharpe
fig.add_trace(
    go.Scatter(x=results_df['MR_Score'], y=results_df['Sharpe'],
               mode='markers', marker=dict(size=10, color=results_df['Composite_Score'],
                                          colorscale='Viridis', showscale=True),
               text=results_df['Portfolio'], showlegend=False),
    row=2, col=1
)

# 4. Maximum drawdowns
fig.add_trace(
    go.Bar(x=results_df['Portfolio'], y=results_df['Max_DD']*100,
           marker=dict(color='darkred'), showlegend=False),
    row=2, col=2
)

# 5. Sparsity (number of assets)
fig.add_trace(
    go.Bar(x=results_df['Portfolio'], y=results_df['N_Assets'],
           marker=dict(color='steelblue'), showlegend=False),
    row=3, col=1
)

# 6. Half-life distribution
fig.add_trace(
    go.Histogram(x=results_df['Half_Life'].clip(0, 100), nbinsx=20,
                 marker=dict(color='purple'), showlegend=False),
    row=3, col=2
)

# Update axes
fig.update_xaxes(tickangle=45, row=1, col=1)
fig.update_xaxes(title_text="Volatility (%)", row=1, col=2)
fig.update_yaxes(title_text="Composite Score", row=1, col=1)
fig.update_yaxes(title_text="Annual Return (%)", row=1, col=2)

fig.update_xaxes(title_text="MR Score", row=2, col=1)
fig.update_xaxes(tickangle=45, row=2, col=2)
fig.update_yaxes(title_text="Sharpe Ratio", row=2, col=1)
fig.update_yaxes(title_text="Max DD (%)", row=2, col=2)

fig.update_xaxes(tickangle=45, row=3, col=1)
fig.update_xaxes(title_text="Half-Life (periods)", row=3, col=2)
fig.update_yaxes(title_text="# Assets", row=3, col=1)
fig.update_yaxes(title_text="Frequency", row=3, col=2)

fig.update_layout(height=1200, title_text="Portfolio Comparison Dashboard")
fig.show()

## 8. Deep Dive: Best Portfolio Analysis

Let's examine the top-ranked portfolio in detail, including:
- Backtesting with transaction costs
- Monte Carlo simulation for robustness
- Trading strategy performance

In [ ]:
# Select best portfolio
best_idx = results_df['Composite_Score'].idxmax()
best_portfolio = portfolio_candidates[best_idx]
best_weights = best_portfolio['weights']
best_info = results_df.loc[best_idx]

print("🏆 BEST PORTFOLIO: " + best_info['Portfolio'])
print("="*80)
print(f"  Method: {best_info['Method']}")
print(f"  Assets: {best_info['N_Assets']}")
print(f"  Composite Score: {best_info['Composite_Score']:.4f}")
print(f"\n📊 Performance Metrics:")
print(f"  Annual Return: {best_info['Annual_Return']*100:.2f}%")
print(f"  Volatility: {best_info['Volatility']*100:.2f}%")
print(f"  Sharpe Ratio: {best_info['Sharpe']:.3f}")
print(f"  Max Drawdown: {best_info['Max_DD']*100:.2f}%")
print(f"  Calmar Ratio: {best_info['Calmar']:.3f}")
print(f"\n🔄 Mean-Reversion Properties:")
print(f"  Hurst Exponent: {best_info['Hurst']:.4f} {'✓ (Mean-Reverting)' if best_info['Hurst'] < 0.5 else '✗'}")
print(f"  ADF p-value: {best_info['ADF_pval']:.6f} {'✓ (Stationary)' if best_info['ADF_pval'] < 0.05 else '✗'}")
print(f"  Half-Life: {best_info['Half_Life']:.2f} periods")
print(f"\n💰 Risk Metrics:")
print(f"  VaR (95%): {best_info['VaR_95']*100:.3f}%")
print(f"  CVaR (95%): {best_info['CVaR_95']*100:.3f}%")
print("="*80)

# Display portfolio composition
weights_best = pd.DataFrame({
    'Asset': asset_names,
    'Weight': best_weights
}).sort_values('Weight', key=abs, ascending=False)

print("\n📋 Portfolio Composition:")
print(weights_best[weights_best['Weight'].abs() > 1e-6].to_string(index=False))

# Compute portfolio value
pf_value = prices_df.values @ best_weights
pf_returns = np.diff(pf_value) / pf_value[:-1]
pf_zscore = (pf_value - np.mean(pf_value)) / np.std(pf_value)

In [ ]:
# Backtest with mean-reversion trading strategy
def backtest_meanrev_strategy(zscore, returns, entry_threshold=2.0, exit_threshold=0.5, 
                                transaction_cost=0.001):
    """
    Backtest pairs trading strategy:
    - Enter when |z| > entry_threshold
    - Exit when |z| < exit_threshold
    - Transaction cost per trade
    """
    position = 0  # 0 = no position, 1 = long, -1 = short
    pnl = np.zeros(len(returns))
    trades = []
    
    for t in range(len(returns)):
        z = zscore[t]
        
        # Entry signals
        if position == 0:
            if z < -entry_threshold:  # Oversold → buy
                position = 1
                pnl[t] = -transaction_cost
                trades.append((t, 'BUY', z))
            elif z > entry_threshold:  # Overbought → sell
                position = -1
                pnl[t] = -transaction_cost
                trades.append((t, 'SELL', z))
        
        # Exit signals
        elif position != 0:
            if abs(z) < exit_threshold:  # Mean reversion → close
                pnl[t] = position * returns[t] - transaction_cost
                position = 0
                trades.append((t, 'CLOSE', z))
            else:
                pnl[t] = position * returns[t]
        
        # Stop loss
        if abs(z) > 4:
            if position != 0:
                pnl[t] += -transaction_cost
                position = 0
                trades.append((t, 'STOP', z))
    
    return pnl, trades

# Run backtest
pnl, trades = backtest_meanrev_strategy(
    pf_zscore[1:], pf_returns, 
    entry_threshold=2.0, 
    exit_threshold=0.5,
    transaction_cost=0.001
)

cum_pnl = np.cumsum(pnl)
total_return = cum_pnl[-1]
sharpe_strat = np.mean(pnl) / np.std(pnl) * np.sqrt(252) if np.std(pnl) > 0 else 0

print(f"\n📈 STRATEGY BACKTEST RESULTS:")
print(f"  Total Return: {total_return*100:.2f}%")
print(f"  Sharpe Ratio: {sharpe_strat:.3f}")
print(f"  Number of Trades: {len([t for t in trades if t[1] in ['BUY', 'SELL']])}")
print(f"  Avg Trade Duration: {len(pnl) / max(len([t for t in trades if t[1] in ['BUY', 'SELL']]), 1):.1f} periods")

# Calculate drawdown
running_max = np.maximum.accumulate(cum_pnl)
drawdown = cum_pnl - running_max
max_dd_strat = np.min(drawdown)
print(f"  Max Drawdown: {max_dd_strat*100:.2f}%")

# Win rate
winning_trades = np.sum(pnl > 0)
total_active_periods = np.sum(pnl != 0)
win_rate = winning_trades / total_active_periods if total_active_periods > 0 else 0
print(f"  Win Rate: {win_rate*100:.1f}%")

In [ ]:
# Visualize strategy performance
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=('Portfolio Price & Z-Score', 'Cumulative Strategy P&L',
                    'Trade Entries/Exits', 'P&L Distribution',
                    'Rolling Sharpe (60-day)', 'Drawdown'),
    specs=[[{"secondary_y": True}, {"type": "scatter"}],
           [{"type": "scatter"}, {"type": "histogram"}],
           [{"type": "scatter"}, {"type": "scatter"}]]
)

# 1. Portfolio price with Z-score overlay
fig.add_trace(
    go.Scatter(y=pf_value, mode='lines', name='Portfolio Price', 
               line=dict(color='blue', width=2)),
    row=1, col=1, secondary_y=False
)
fig.add_trace(
    go.Scatter(y=pf_zscore, mode='lines', name='Z-Score', 
               line=dict(color='red', width=1)),
    row=1, col=1, secondary_y=True
)
# Trading bands
fig.add_hline(y=2, line_dash="dash", line_color="green", secondary_y=True, row=1, col=1)
fig.add_hline(y=-2, line_dash="dash", line_color="green", secondary_y=True, row=1, col=1)
fig.add_hline(y=0, line_dash="solid", line_color="gray", secondary_y=True, row=1, col=1)

# 2. Cumulative P&L
fig.add_trace(
    go.Scatter(y=cum_pnl * 100, mode='lines', name='Cum P&L',
               line=dict(color='darkgreen', width=2), fill='tozeroy', showlegend=False),
    row=1, col=2
)

# 3. Trade markers
buy_trades = [t for t in trades if t[1] == 'BUY']
sell_trades = [t for t in trades if t[1] == 'SELL']
close_trades = [t for t in trades if t[1] == 'CLOSE']

if buy_trades:
    fig.add_trace(
        go.Scatter(x=[t[0] for t in buy_trades], 
                   y=[pf_value[t[0]] for t in buy_trades],
                   mode='markers', marker=dict(symbol='triangle-up', size=12, color='green'),
                   name='Buy', showlegend=True),
        row=2, col=1
    )

if sell_trades:
    fig.add_trace(
        go.Scatter(x=[t[0] for t in sell_trades], 
                   y=[pf_value[t[0]] for t in sell_trades],
                   mode='markers', marker=dict(symbol='triangle-down', size=12, color='red'),
                   name='Sell', showlegend=True),
        row=2, col=1
    )

if close_trades:
    fig.add_trace(
        go.Scatter(x=[t[0] for t in close_trades], 
                   y=[pf_value[t[0]] for t in close_trades],
                   mode='markers', marker=dict(symbol='x', size=10, color='blue'),
                   name='Close', showlegend=True),
        row=2, col=1
    )

fig.add_trace(
    go.Scatter(y=pf_value, mode='lines', name='Price', 
               line=dict(color='lightgray', width=1), showlegend=False),
    row=2, col=1
)

# 4. P&L distribution
fig.add_trace(
    go.Histogram(x=pnl * 100, nbinsx=50, name='P&L', showlegend=False),
    row=2, col=2
)

# 5. Rolling Sharpe
rolling_window = 60
rolling_sharpe = []
for i in range(rolling_window, len(pnl)):
    window_pnl = pnl[i-rolling_window:i]
    rs = np.mean(window_pnl) / np.std(window_pnl) * np.sqrt(252) if np.std(window_pnl) > 0 else 0
    rolling_sharpe.append(rs)

fig.add_trace(
    go.Scatter(y=rolling_sharpe, mode='lines', name='Rolling Sharpe',
               line=dict(color='purple', width=2), showlegend=False),
    row=3, col=1
)
fig.add_hline(y=0, line_dash="solid", line_color="black", row=3, col=1)

# 6. Drawdown
fig.add_trace(
    go.Scatter(y=drawdown * 100, mode='lines', name='Drawdown',
               line=dict(color='red', width=2), fill='tozeroy', showlegend=False),
    row=3, col=2
)

# Update axes
fig.update_yaxes(title_text="Price", secondary_y=False, row=1, col=1)
fig.update_yaxes(title_text="Z-Score", secondary_y=True, row=1, col=1)
fig.update_yaxes(title_text="Cum P&L (%)", row=1, col=2)
fig.update_yaxes(title_text="Price", row=2, col=1)
fig.update_yaxes(title_text="Frequency", row=2, col=2)
fig.update_yaxes(title_text="Sharpe", row=3, col=1)
fig.update_yaxes(title_text="DD (%)", row=3, col=2)

fig.update_layout(height=1200, title_text=f"Strategy Performance: {best_info['Portfolio']}")
fig.show()

## 9. Monte Carlo Robustness Analysis

Test portfolio robustness through:
1. **Bootstrap resampling** of historical returns
2. **Parameter sensitivity** analysis
3. **Stress testing** under extreme scenarios

In [ ]:
# Monte Carlo simulation with bootstrap
n_simulations = 1000
sim_sharpes = []
sim_returns = []
sim_drawdowns = []

np.random.seed(42)

print(f"Running {n_simulations} Monte Carlo simulations...")

for sim in range(n_simulations):
    # Bootstrap resample returns
    idx = np.random.choice(len(pf_returns), size=len(pf_returns), replace=True)
    sim_rets = pf_returns[idx]
    
    # Calculate metrics
    ann_ret = np.mean(sim_rets) * 252
    vol = np.std(sim_rets) * np.sqrt(252)
    sharpe = ann_ret / vol if vol > 0 else 0
    
    # Drawdown
    cum_rets = np.cumprod(1 + sim_rets)
    running_max = np.maximum.accumulate(cum_rets)
    dd = (cum_rets - running_max) / running_max
    max_dd = np.min(dd)
    
    sim_sharpes.append(sharpe)
    sim_returns.append(ann_ret)
    sim_drawdowns.append(max_dd)

# Summary statistics
print(f"\n📊 Monte Carlo Results ({n_simulations} simulations):")
print(f"  Sharpe Ratio:")
print(f"    Mean: {np.mean(sim_sharpes):.3f}")
print(f"    Median: {np.median(sim_sharpes):.3f}")
print(f"    5th percentile: {np.percentile(sim_sharpes, 5):.3f}")
print(f"    95th percentile: {np.percentile(sim_sharpes, 95):.3f}")
print(f"\n  Annual Return:")
print(f"    Mean: {np.mean(sim_returns)*100:.2f}%")
print(f"    5th percentile: {np.percentile(sim_returns, 5)*100:.2f}%")
print(f"    95th percentile: {np.percentile(sim_returns, 95)*100:.2f}%")
print(f"\n  Max Drawdown:")
print(f"    Mean: {np.mean(sim_drawdowns)*100:.2f}%")
print(f"    5th percentile: {np.percentile(sim_drawdowns, 5)*100:.2f}%")
print(f"    95th percentile: {np.percentile(sim_drawdowns, 95)*100:.2f}%")

# Probability of positive Sharpe
prob_positive = np.mean(np.array(sim_sharpes) > 0)
print(f"\n  Probability of positive Sharpe: {prob_positive*100:.1f}%")

In [ ]:
# Visualize Monte Carlo results
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Sharpe Ratio Distribution', 'Return vs Risk',
                    'Max Drawdown Distribution', 'Confidence Intervals'),
    specs=[[{"type": "histogram"}, {"type": "scatter"}],
           [{"type": "histogram"}, {"type": "box"}]]
)

# 1. Sharpe distribution
fig.add_trace(
    go.Histogram(x=sim_sharpes, nbinsx=50, name='Sharpe', showlegend=False),
    row=1, col=1
)
fig.add_vline(x=np.mean(sim_sharpes), line_dash="dash", line_color="red", row=1, col=1)
fig.add_vline(x=0, line_dash="solid", line_color="black", row=1, col=1)

# 2. Return vs risk scatter
fig.add_trace(
    go.Scatter(x=np.array(sim_returns)*np.sqrt(252), y=np.array(sim_returns),
               mode='markers', marker=dict(size=4, color=sim_sharpes, 
                                          colorscale='RdYlGn', showscale=True),
               showlegend=False),
    row=1, col=2
)

# 3. Drawdown distribution
fig.add_trace(
    go.Histogram(x=np.array(sim_drawdowns)*100, nbinsx=50, showlegend=False),
    row=2, col=1
)
fig.add_vline(x=np.mean(sim_drawdowns)*100, line_dash="dash", line_color="red", row=2, col=1)

# 4. Box plots for key metrics
metrics_data = [
    go.Box(y=sim_sharpes, name='Sharpe', marker=dict(color='blue')),
    go.Box(y=np.array(sim_returns)*100, name='Return (%)', marker=dict(color='green')),
    go.Box(y=np.array(sim_drawdowns)*100, name='Max DD (%)', marker=dict(color='red'))
]

for metric in metrics_data:
    fig.add_trace(metric, row=2, col=2)

# Update axes
fig.update_xaxes(title_text="Sharpe Ratio", row=1, col=1)
fig.update_xaxes(title_text="Volatility", row=1, col=2)
fig.update_xaxes(title_text="Max DD (%)", row=2, col=1)
fig.update_yaxes(title_text="Frequency", row=1, col=1)
fig.update_yaxes(title_text="Ann. Return", row=1, col=2)
fig.update_yaxes(title_text="Frequency", row=2, col=1)

fig.update_layout(height=800, title_text=f"Monte Carlo Robustness Analysis (N={n_simulations})")
fig.show()

## 10. Production Implementation Guide

### Deployment Considerations

**1. Data Requirements:**
- Minimum history: 252 trading days (1 year)
- Update frequency: Daily for portfolios, intraday for signals
- Data quality: Clean prices, corporate action adjusted

**2. Rebalancing Protocol:**

The portfolio weights should be recalculated periodically:
$$T_{\text{rebal}} = \max\left(\frac{\text{half-life}}{2}, 20 \text{ days}\right)$$

**Trigger conditions for early rebalancing:**
- Hurst exponent changes by >0.1
- ADF p-value exceeds 0.10 (losing stationarity)
- Individual asset weight drifts >50% from target
- Max drawdown exceeds threshold (e.g., -15%)

**3. Position Sizing:**

Total capital allocation:
$$\text{Position Size} = \frac{\text{Capital}}{\sigma_{\text{portfolio}} \cdot \text{Volatility Target}}$$

where Volatility Target = 10-15% annualized (typical for stat arb)

**4. Risk Management Rules:**

- **Stop Loss:** Exit if $|z\text{-score}| > 4$
- **Time Stop:** Exit if position held > $2 \times \text{half-life}$ without mean-reversion
- **Correlation Break:** Exit if asset correlation matrix changes significantly (>30% from training)
- **Liquidity Check:** Ensure each asset ADV > 10× target position size

**5. Transaction Cost Model:**

Total cost per trade:
$$C_{\text{total}} = C_{\text{spread}} + C_{\text{commission}} + C_{\text{impact}}$$

where:
- Spread cost: $\approx 0.5 \times \text{bid-ask spread}$
- Commission: Broker-dependent (0.1-1 bps)
- Market impact: $\approx 0.1 \times \sqrt{\frac{\text{order size}}{\text{ADV}}}$

**6. Performance Monitoring:**

Daily metrics to track:
- Sharpe ratio (rolling 60-day)
- Current z-score and distance from mean
- Half-life stability (rolling 120-day estimate)
- Correlation matrix stability (Frobenius norm of difference)

In [ ]:
# Generate production-ready trading signals
def generate_trading_signals(zscore, entry_threshold=2.0, exit_threshold=0.5):
    """
    Generate actionable trading signals
    Returns: signal array (-1: short, 0: flat, 1: long)
    """
    signals = np.zeros(len(zscore))
    position = 0
    
    for t in range(len(zscore)):
        z = zscore[t]
        
        if position == 0:
            if z < -entry_threshold:
                position = 1
            elif z > entry_threshold:
                position = -1
        else:
            if abs(z) < exit_threshold:
                position = 0
            elif abs(z) > 4:  # Stop loss
                position = 0
        
        signals[t] = position
    
    return signals

signals = generate_trading_signals(pf_zscore, entry_threshold=2.0, exit_threshold=0.5)

# Calculate position sizing
volatility_target = 0.12  # 12% annual vol target
portfolio_vol = np.std(pf_returns) * np.sqrt(252)
position_scalar = volatility_target / portfolio_vol if portfolio_vol > 0 else 1.0

print(f"📊 PRODUCTION TRADING PARAMETERS")
print(f"="*80)
print(f"\n💼 Position Sizing:")
print(f"  Portfolio Volatility: {portfolio_vol*100:.2f}%")
print(f"  Volatility Target: {volatility_target*100:.2f}%")
print(f"  Position Scalar: {position_scalar:.3f}x")
print(f"  Recommended allocation: {position_scalar*100:.1f}% of capital")

# Signal statistics
long_signals = np.sum(signals == 1)
short_signals = np.sum(signals == -1)
flat_signals = np.sum(signals == 0)
total_signals = len(signals)

print(f"\n📈 Signal Distribution:")
print(f"  Long:  {long_signals:4d} periods ({100*long_signals/total_signals:.1f}%)")
print(f"  Short: {short_signals:4d} periods ({100*short_signals/total_signals:.1f}%)")
print(f"  Flat:  {flat_signals:4d} periods ({100*flat_signals/total_signals:.1f}%)")

# Rebalancing frequency
rebal_freq = max(best_info['Half_Life'] / 2, 20)
print(f"\n🔄 Rebalancing:")
print(f"  Recommended frequency: {rebal_freq:.0f} days")
print(f"  Based on half-life: {best_info['Half_Life']:.1f} periods")

# Risk limits
print(f"\n⚠️ Risk Limits:")
print(f"  Max z-score (stop loss): ±4.0")
print(f"  Time stop (no reversion): {2*best_info['Half_Life']:.0f} periods")
print(f"  Max drawdown alert: -15%")
print(f"  Min Hurst for continued trading: {best_info['Hurst'] - 0.1:.3f}")
print(f"  Max ADF p-value: 0.10")

# Transaction cost estimate
avg_turnover = np.sum(np.abs(np.diff(signals))) / len(signals)
annual_trades = avg_turnover * 252
est_txn_cost = annual_trades * 0.001  # 10 bps per trade

print(f"\n💰 Transaction Costs:")
print(f"  Estimated annual trades: {annual_trades:.0f}")
print(f"  Assumed cost per trade: 0.10%")
print(f"  Total annual cost: {est_txn_cost*100:.2f}%")
print(f"  Net expected return: {(best_info['Annual_Return'] - est_txn_cost)*100:.2f}%")

print(f"\n" + "="*80)

In [ ]:
# Final visualization: Production Dashboard
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=('Current Portfolio Weights', 'Signal Timeline',
                    'Z-Score with Trading Bands', 'Expected vs Actual Returns',
                    'Rolling Metrics', 'Risk Dashboard'),
    specs=[[{"type": "bar"}, {"type": "scatter"}, {"type": "scatter"}],
           [{"type": "scatter"}, {"type": "scatter"}, {"type": "indicator"}]]
)

# 1. Portfolio weights (pie chart alternative)
active_w = best_weights[np.abs(best_weights) > 1e-6]
active_names = [asset_names[i] for i, w in enumerate(best_weights) if abs(w) > 1e-6]
colors_pf = ['green' if w > 0 else 'red' for w in active_w]

fig.add_trace(
    go.Bar(x=active_names, y=active_w, marker=dict(color=colors_pf), showlegend=False),
    row=1, col=1
)

# 2. Signals over time
signal_colors = ['red' if s == -1 else ('green' if s == 1 else 'gray') for s in signals]
fig.add_trace(
    go.Scatter(y=signals, mode='lines', line=dict(color='blue', width=1),
               fill='tozeroy', showlegend=False),
    row=1, col=2
)

# 3. Z-score with bands
fig.add_trace(
    go.Scatter(y=pf_zscore, mode='lines', name='Z-Score', 
               line=dict(color='darkblue', width=2), showlegend=False),
    row=1, col=3
)
fig.add_hline(y=2, line_dash="dash", line_color="red", annotation_text="Entry", row=1, col=3)
fig.add_hline(y=-2, line_dash="dash", line_color="red", row=1, col=3)
fig.add_hline(y=0.5, line_dash="dot", line_color="green", annotation_text="Exit", row=1, col=3)
fig.add_hline(y=-0.5, line_dash="dot", line_color="green", row=1, col=3)

# 4. Expected vs Actual (scatter of daily returns vs z-score)
fig.add_trace(
    go.Scatter(x=pf_zscore[1:], y=pf_returns*100, mode='markers',
               marker=dict(size=4, color=pf_returns, colorscale='RdYlGn'),
               showlegend=False),
    row=2, col=1
)
# Add regression line
if len(pf_zscore[1:]) > 0 and np.std(pf_zscore[1:]) > 1e-6:
    z_fit = np.polyfit(pf_zscore[1:], pf_returns, 1)
    z_line = z_fit[0] * np.sort(pf_zscore[1:]) + z_fit[1]
    fig.add_trace(
        go.Scatter(x=np.sort(pf_zscore[1:]), y=z_line*100, mode='lines',
                   line=dict(color='red', dash='dash'), showlegend=False),
        row=2, col=1
    )

# 5. Rolling Sharpe and Max DD
window = 60
rolling_ret = pd.Series(pf_returns).rolling(window).mean() * 252
rolling_vol = pd.Series(pf_returns).rolling(window).std() * np.sqrt(252)
rolling_sr = (rolling_ret / rolling_vol).fillna(0)

fig.add_trace(
    go.Scatter(y=rolling_sr, mode='lines', name='60d Sharpe',
               line=dict(color='purple', width=2), showlegend=False),
    row=2, col=2
)
fig.add_hline(y=1.0, line_dash="dash", line_color="green", row=2, col=2)

# 6. Key metrics (gauge/indicator style)
# Using scatter as placeholder for indicator
current_sharpe = best_info['Sharpe']
current_dd = best_info['Max_DD']
current_hurst = best_info['Hurst']

metrics_text = f"""
Current Status:
Sharpe: {current_sharpe:.2f}
Max DD: {current_dd*100:.1f}%
Hurst: {current_hurst:.3f}
Half-Life: {best_info['Half_Life']:.0f}d
Status: {'✓ ACTIVE' if current_sharpe > 0.5 and abs(current_dd) < 0.2 else '⚠ REVIEW'}
"""

fig.add_annotation(
    text=metrics_text,
    xref="x6", yref="y6",
    x=0.5, y=0.5,
    showarrow=False,
    font=dict(size=14, family="monospace"),
    align="left",
    row=2, col=3
)

# Update axes
fig.update_xaxes(tickangle=45, row=1, col=1)
fig.update_yaxes(title_text="Weight", row=1, col=1)
fig.update_yaxes(title_text="Signal", row=1, col=2)
fig.update_yaxes(title_text="Z-Score", row=1, col=3)
fig.update_xaxes(title_text="Z-Score", row=2, col=1)
fig.update_yaxes(title_text="Return (%)", row=2, col=1)
fig.update_yaxes(title_text="Sharpe", row=2, col=2)

fig.update_layout(height=900, title_text="🚀 Production Trading Dashboard", showlegend=False)
fig.show()

print(f"\n✅ Notebook complete! Portfolio ready for production deployment.")

## 11. Summary & Key Takeaways

### What We've Accomplished

1. **Mathematical Foundation** ✓
   - Derived and explained Sparse PCA with L1 regularization
   - Detailed Box & Tao decomposition (Low-rank + Sparse + Noise)
   - Complete Hurst exponent theory via R/S analysis
   - Elastic Net cointegration formulation

2. **Multiple Portfolio Construction Methods** ✓
   - Sparse PCA: Variance-maximizing sparse portfolios
   - Box & Tao: Idiosyncratic component extraction
   - Hurst-based: Direct mean-reversion targeting
   - Cointegration: Statistically stationary portfolios

3. **Comprehensive Evaluation** ✓
   - Multi-criteria scoring system
   - Risk-adjusted performance metrics (Sharpe, Calmar, VaR, CVaR)
   - Mean-reversion quality assessment
   - Sparsity and transaction cost considerations

4. **Real-World Application** ✓
   - Backtesting with transaction costs
   - Monte Carlo robustness analysis
   - Position sizing and risk management
   - Production deployment guidelines

### Key Insights

**Best Practices:**
- Prefer portfolios with Hurst < 0.45 and ADF p-value < 0.05
- Target 5-10 assets for optimal sparsity/diversification tradeoff
- Rebalance at intervals of ~half-life/2 (typically 10-30 days)
- Use 2σ entry, 0.5σ exit, 4σ stop-loss thresholds
- Size positions for 10-15% annualized volatility

**Common Pitfalls:**
- Over-fitting: Use walk-forward validation in production
- Regime changes: Monitor rolling Hurst and correlation stability
- Transaction costs: Can erode 2-5% annually for frequent rebalancing
- Liquidity: Ensure ADV > 10× position size for each asset

### Further Reading

- d'Aspremont (2011): "Identifying Small Mean Reverting Portfolios"
- Candès et al. (2011): "Robust Principal Component Analysis?"
- Zou & Hastie (2005): "Regularization and Variable Selection via Elastic Net"
- Hurst (1951): "Long-term Storage Capacity of Reservoirs"

---

**Next Steps:**
- Test on real market data (ETFs, stocks, crypto)
- Implement walk-forward optimization
- Add regime detection (HMM/Markov switching)
- Integrate with live trading infrastructure

## 2. Load Real-World Data

We'll use real market data to demonstrate the algorithms. For this example, we'll generate synthetic data that mimics real market behavior with:
- Common factor structure (market beta)
- Idiosyncratic mean-reverting components
- Realistic noise levels

### 1.2 Box & Tao Decomposition (Robust PCA)

Decomposes the price matrix into three components:
$$X = L + S + N$$

where:
- $L$ = **low-rank** component (common market factors)
- $S$ = **sparse** component (idiosyncratic mean-reverting opportunities) ← **Our target!**
- $N$ = noise

**Optimization Problem:**
$$\min_{L,S} \quad \|L\|_* + \lambda \|S\|_1 \quad \text{s.t.} \quad X = L + S + N, \quad \|N\|_F \leq \epsilon$$

where:
- $\|L\|_* = \sum_i \sigma_i(L)$ = nuclear norm (sum of singular values)
- $\|\cdot\|_1$ = L1 norm (sum of absolute values)
- $\|\cdot\|_F$ = Frobenius norm

**Algorithm (ADMM - Alternating Direction Method of Multipliers):**
1. Initialize $L = S = 0$, $Y = 0$ (dual variable)
2. Repeat:
   - Update $L$: Soft-threshold singular values
   - Update $S$: Soft-threshold entries  
   - Update $Y$: $Y \leftarrow Y + \rho(X - L - S)$

**Interpretation:** The sparse component $S$ reveals assets with idiosyncratic behavior not explained by common factors → potential mean-reversion candidates.

In [ ]:
# Import libraries
import sys
sys.path.append('/Users/melvinalvarez/Documents/Enki/Workspace/rust-arblab')

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Import our sparse mean-reversion module
from python.sparse_meanrev import (
    sparse_pca, box_tao_decomposition, hurst_exponent, 
    sparse_cointegration, generate_sparse_meanrev_signals,
    RUST_AVAILABLE
)

print(f"✓ Libraries imported successfully!")
print(f"✓ Rust acceleration: {'ENABLED ⚡' if RUST_AVAILABLE else 'DISABLED'}")